In [1]:
import json
import pandas as pd
import os
import sys
import numpy as np
from sklearn.cluster import KMeans, MiniBatchKMeans


# Get the parent directory of the current directory
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the parent directory to the Python path
sys.path.append(parent_dir)
from config import Config

In [2]:
config = Config()
data_path = '../' + config.data_path

In [3]:
train = pd.read_json(data_path + 'train.json')
dev = pd.read_json(data_path + 'dev.json')

In [4]:
train['tokenized_instructions'] = train['Instructions'].map(lambda x: config.nlp(x))
dev['tokenized_instructions'] = dev['Instructions'].map(lambda x: config.nlp(x))

In [5]:
def document_embedding(doc):
    return np.average([t.vector for t in doc], axis=0)
    
train['embeddings'] = train['tokenized_instructions'].map(lambda x: document_embedding(x))
dev['embeddings'] = dev['tokenized_instructions'].map(lambda x: document_embedding(x))

In [6]:
train['embeddings'] = train['tokenized_instructions'].map(lambda x: document_embedding(x))

In [7]:
train_embeddings = np.array([d for d in train['embeddings']])
dev_embeddings= np.array([d for d in dev['embeddings']])

In [8]:
train_embeddings.shape

(7801, 300)

In [9]:
kmeans = MiniBatchKMeans(n_clusters=5,random_state=0,batch_size=8,n_init="auto")

In [10]:
kmeans.fit(train_embeddings)

/Users/felix/opt/anaconda3/envs/weak-ingredient-recognition/lib/python3.9/site-packages/threadpoolctl.py:1010: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


MiniBatchKMeans(batch_size=8, n_clusters=5, n_init='auto', random_state=0)

In [13]:
kmeans.cluster_centers_[0].shape

(300,)